# CNN Sign Language Classifier

## Importing Libraries and dataset

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_images = pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
test_images = pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

In [ ]:
Y_train = train_images['label'].values
Y_test = test_images['label'].values

X_train = train_images.drop(["label"],axis=1).values
X_test = test_images.drop(["label"],axis=1).values

In [ ]:
X_train = X_train.reshape(-1,28,28)
X_test = X_test.reshape(-1,28,28)
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i],cmap=plt.get_cmap('gray'))
    plt.xlabel(Y_train[i])
plt.show()

## Reshaping, Normalisation and Encoding

In [ ]:
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

In [ ]:
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
X_train.shape

In [ ]:
label_binrizer = LabelBinarizer()
Y_train = label_binrizer.fit_transform(Y_train)
Y_test = label_binrizer.fit_transform(Y_test)

## Image augmentation

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=30, zoom_range=0.2, horizontal_flip = True,
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2,
                                   fill_mode='nearest')

In [ ]:
validation_datagen = ImageDataGenerator()

## Model

In [ ]:
model = Sequential()
model.add(Conv2D(filters= 32 , kernel_size=(3,3), padding = 'same', activation = 'relu' , input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size=(2,2), strides = 2, padding = 'same'))
model.add(BatchNormalization())

model.add(Conv2D(filters=32, kernel_size=(3,3) , padding = 'same', activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2), strides = 2))
model.add(BatchNormalization())

model.add(Conv2D(filters=64, kernel_size=(3,3), padding = 'same', activation = 'relu'))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding = 'valid', activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2) , strides = 2))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())

model.add(Dense(24, activation = 'softmax'))

In [ ]:
model.compile(optimizer= 'adam' ,loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.summary()

In [ ]:
history = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=32),
                              steps_per_epoch=len(X_train) // 32,
                              epochs=10,
                              validation_data=validation_datagen.flow(X_test, Y_test, batch_size=32),
                              validation_steps=len(X_test) // 32)

## Evaluate

In [ ]:
model.evaluate(X_test, Y_test, verbose=0)

In [ ]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()

Don't forget to upvote, if you found my notebook useful.